# Impact of Urban Revitalization In Birmingham Alabama

I returned to Birmingham almost exactly 20 years ago to find that the downtown area was undergoing an amazing transformation.  I will use some of the skills I've developed in my Data Science coursework to analyze this transformation.  My hope is that I can show the progress and provide some insight as to where more progress can be made.

## Methodology

I will use the Foursquare API to identify clusters of highly trafficed venues.  I will use geographic data from https://data.birminghamal.gov to break the City of Birmingham up into its individual neighborhoods and rank the types of venues available.  Along with the Foursquare data I will use data from https://www.bhamwiki.com/w/Operation_New_Birmingham and the Birmingham Business Journal https://www.bizjournals.com/birmingham/ to tie together the venue clusters with revitalization efforts.  I will then use k-means clustering to attempt to classify groups of neighborhoods based on venue categories contained in each neighborhood.  My theory is that the composition of venues within the venue clusters should equate to the revitalization efforts. Ultimately I hope to see locally owned businesses thriving in these revitalized zones.  What follows are the detailed steps I performed during this analysis.

### Here I will install and import the Python libraries I need for the project

In [1]:
!pip install beautifulsoup4
!pip install folium
!pip install geopy
!pip install geopandas

In [2]:
import requests
import pandas as pd
import geopandas
import numpy as np
from bs4 import BeautifulSoup
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library
from geopy.geocoders import Nominatim



## Map of Birmingham Alabama

In [3]:
address = 'Birmingham USA'

geolocator = Nominatim(user_agent="bhm_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Birmingham, AL are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Birmingham, AL are 33.5206824, -86.8024326.


In [4]:
map_bhm = folium.Map(location=[latitude, longitude], zoom_start=12,
                     zoom_control=False,
               scrollWheelZoom=False,
               dragging=False)
map_bhm


## Dataframe for the Neighborhoods of Birmingham

I used geographic data from the Birmingam Planning and Engineering department made available on their Open Data initiative website https://data.birmihnghamal.gov/organization/birmingham-planning-engineering where I was able to get a geoJSON file containing data for me to understand the geography of the neighborhoods.  I ultimately used the Python GeoPandas library to build a dataframe containing the neighborhood geometry.  I then found a central latitude and longitude for each neighborhood that would allow me to make a FourSquare api call for each neighborhood.  The GeoJSON file was uploaded into my IBM Watson Studio Jupyter Notebook.  I read this into a stream that I passed into a geopandas dataframe constructor.

In [5]:
# The code was removed by Watson Studio for sharing.

In [6]:
bhm_neighborhoods_geo = geopandas.read_file(streaming_body_1, layer="features")

In [7]:
bhm_neighborhoods_geo.head(50)

,FID,NUM,NAME,Shape_Leng,Shape_Area,geometry
0,1,701,Apple Valley,57024.688015,2.008130e+07,"POLYGON ((-86.68047 33.67689, -86.68031 33.676..."
1,2,802,Sherman Heights,452319.727019,2.736706e+08,"POLYGON ((-87.01275 33.49449, -87.01340 33.494..."
2,3,705,Sun Valley,36022.229835,1.988828e+07,"MULTIPOLYGON (((-86.70463 33.64090, -86.70817 ..."
3,4,1003,Spring Lake,97411.325741,5.917413e+07,"POLYGON ((-86.65253 33.60168, -86.65271 33.601..."
4,5,703,Echo Highlands,111881.645356,1.023235e+08,"MULTIPOLYGON (((-86.71446 33.63031, -86.71515 ..."
5,6,1105,Hooper City,147831.568931,1.942805e+08,"POLYGON ((-86.84833 33.54339, -86.84837 33.543..."
6,7,1002,Killough Springs,64441.881010,4.599536e+07,"POLYGON ((-86.71876 33.60451, -86.71879 33.604..."
7,8,702,Bridlewood,36919.271552,8.493263e+06,"MULTIPOLYGON (((-86.70043 33.62115, -86.70182 ..."
8,9,1004,Liberty Highlands,252902.147375,1.787171e+08,"POLYGON ((-86.67412 33.54694, -86.67412 33.546..."
9,10,1001,Huffman,68312.939683,8.350917e+07,"POLYGON ((-86.67150 33.58350, -86.67154 33.583..."


Once I had the geometry for each neighborhood, I was able to find a central latitude and longitude.  I added those columns to the dataframe.

In [8]:
bhm_neighborhoods_geo['lng'] = bhm_neighborhoods_geo['geometry'].centroid.x
bhm_neighborhoods_geo['lat'] = bhm_neighborhoods_geo['geometry'].centroid.y
    
bhm_neighborhoods_geo.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  if __name__ == '__main__':
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  from ipykernel import kernelapp as app


,FID,NUM,NAME,Shape_Leng,Shape_Area,geometry,lng,lat
0,1,701,Apple Valley,57024.688015,2.008130e+07,"POLYGON ((-86.68047 33.67689, -86.68031 33.676...",-86.686467,33.666536
1,2,802,Sherman Heights,452319.727019,2.736706e+08,"POLYGON ((-87.01275 33.49449, -87.01340 33.494...",-87.024548,33.561900
2,3,705,Sun Valley,36022.229835,1.988828e+07,"MULTIPOLYGON (((-86.70463 33.64090, -86.70817 ...",-86.705592,33.653732
3,4,1003,Spring Lake,97411.325741,5.917413e+07,"POLYGON ((-86.65253 33.60168, -86.65271 33.601...",-86.668153,33.616183
4,5,703,Echo Highlands,111881.645356,1.023235e+08,"MULTIPOLYGON (((-86.71446 33.63031, -86.71515 ...",-86.724420,33.630341


## Get Venues from Foursquare

Now that we have the latitude and longitude for a point in each neighborhood, we'll call the foursquare api for each neighborood in the dataframe.

In [9]:
# The code was removed by Watson Studio for sharing.

In [10]:
radius = 3250
venues_list=[]


            
# make the GET request for each of the bhm neighborhood 
for index, row in bhm_neighborhoods_geo.iterrows():
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            row['lat'], 
            row['lng'], 
            radius, 
            LIMIT)
    try:
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
                row['FID'],        
                row['NAME'],
                v['venue']['name'],            
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],
                v['venue']['location']['distance'],
                v['venue']['categories'][0]['name']) for v in results])
    except KeyError:
        print("key error")


nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['FID',
                         'Neighborhood',
                         'Venue',                          
                          'Venue Latitude', 
                          'Venue Longitude',
                          'Venue Distance',
                          'Venue Category']
nearby_venues






,FID,Neighborhood,Venue,Venue Latitude,Venue Longitude,Venue Distance,Venue Category
0,1,Apple Valley,Ken's Hickory Pit Barbecue,33.662730,-86.706574,1910,BBQ Joint
1,1,Apple Valley,Green Acres,33.649504,-86.683317,1918,Southern / Soul Food Restaurant
2,1,Apple Valley,Pizza Hut,33.647932,-86.684552,2078,Pizza Place
3,1,Apple Valley,Jack's,33.689013,-86.683100,2521,Burger Joint
4,1,Apple Valley,US Post Office,33.648344,-86.681614,2074,Post Office
...,...,...,...,...,...,...,...
5791,99,East Brownville,Wiggins Park & Recreation Center,33.463692,-86.885846,2910,Park
5792,99,East Brownville,High Line Ore Trail,33.453498,-86.925759,2955,Trail
5793,99,East Brownville,Graybar Electric Supply,33.442222,-86.866392,2995,Construction & Landscaping
5794,99,East Brownville,Western Hills Mall,33.466045,-86.912681,3206,Shopping Mall


## Encoding the Data for Analysis
Our data is in categorical terms, we need a way to convert the data into a form where we can analyze it with a machine learning algorithm.  One way to do this is to create a matrix of the categories and neighborhoods such that an intersecting occurence of category and neighborhood is marked with a 1 all other elements are 0.  This will be useful in grouping the data in a subsequent step.

In [11]:
# one hot encoding
bhm_onehot = pd.get_dummies(nearby_venues[['Venue Category']], prefix="", prefix_sep="")
#add the neighborhood column back in 
bhm_onehot['Neighborhood'] = nearby_venues['Neighborhood']

#lets get this column moved over so the resulting table looks a little better
fixed_columns = [bhm_onehot.columns[-1]] + list(bhm_onehot.columns[:-1])
bhm_onehot = bhm_onehot[fixed_columns]

#lets take a look a the first 20 rows
bhm_onehot.head(20)

,Zoo,Accessories Store,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Arcade,...,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Weight Loss Center,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Grouping the Venue Data
Now I will take the matrix of venue data and perform a "group by" operation so that we have a single entry for each neighborhood.  That single entry will have a tally of each venue catgegory.

In [12]:
bhm_grouped = bhm_onehot.groupby('Neighborhood').sum().reset_index()
bhm_grouped

,Neighborhood,Zoo,Accessories Store,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,...,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Weight Loss Center,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Acipco-Finley,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Airport Highlands,0,0,2,0,3,1,2,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Apple Valley,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,Arlington - West End,0,0,0,0,0,0,2,0,0,...,1,1,0,0,0,0,0,1,0,0
4,Belview Heights,0,0,0,0,0,0,5,0,0,...,2,0,0,1,0,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,West Goldwire,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
94,Woodland Park,0,0,0,0,0,0,1,0,1,...,1,1,1,0,0,0,0,2,0,0
95,Woodlawn,0,0,1,1,3,2,0,0,1,...,0,0,0,0,0,0,0,1,0,0
96,Wylam,0,0,0,0,0,0,2,0,0,...,1,0,0,0,0,0,0,1,0,0


## Top Venues for Each Neighborhood
We'll step through each neighborhood here and rank the venue types by occurence so that we get a picture of what is popular in each neighborhood.

In [13]:
num_top_venues = 5

for hood in bhm_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bhm_grouped[bhm_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Acipco-Finley----
                             venue  freq
0              Fried Chicken Joint   4.0
1                   Discount Store   3.0
2  Southern / Soul Food Restaurant   2.0
3                      Gas Station   2.0
4                     Intersection   2.0


----Airport Highlands----
                  venue  freq
0   Rental Car Location   6.0
1  Fast Food Restaurant   6.0
2       Airport Service   3.0
3   American Restaurant   2.0
4                  Park   2.0


----Apple Valley----
                  venue  freq
0           Gas Station   3.0
1              Pharmacy   3.0
2        Discount Store   3.0
3  Fast Food Restaurant   3.0
4             BBQ Joint   2.0


----Arlington - West End----
                  venue  freq
0        Discount Store   9.0
1  Fast Food Restaurant   8.0
2              Pharmacy   5.0
3         Grocery Store   4.0
4   Fried Chicken Joint   4.0


----Belview Heights----
                  venue  freq
0  Fast Food Restaurant  12.0
1        Discount Store 

## Presentation of Top Venues for Each Neighborhood
Let's pull this together into a nice table format by getting the data into a Pandas dataframe.

In [14]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [15]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bhm_grouped['Neighborhood']

for ind in np.arange(bhm_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bhm_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Acipco-Finley,Fried Chicken Joint,Discount Store,Fast Food Restaurant,Southern / Soul Food Restaurant,Intersection,Gas Station,Theater,Café,Brazilian Restaurant,Cosmetics Shop
1,Airport Highlands,Fast Food Restaurant,Rental Car Location,Airport Service,Café,Airport,American Restaurant,Park,Coffee Shop,Storage Facility,Fried Chicken Joint
2,Apple Valley,Discount Store,Pharmacy,Gas Station,Fast Food Restaurant,Breakfast Spot,Pizza Place,BBQ Joint,Sandwich Place,Southern / Soul Food Restaurant,Bookstore
3,Arlington - West End,Discount Store,Fast Food Restaurant,Pharmacy,Grocery Store,Fried Chicken Joint,Gas Station,Park,Sandwich Place,Pizza Place,Chinese Restaurant
4,Belview Heights,Fast Food Restaurant,Discount Store,Fried Chicken Joint,Gas Station,American Restaurant,Grocery Store,Pharmacy,Southern / Soul Food Restaurant,Sandwich Place,Pizza Place
...,...,...,...,...,...,...,...,...,...,...,...
93,West Goldwire,Gas Station,Trail,Grocery Store,Construction & Landscaping,Sandwich Place,Dog Run,Bar,Fried Chicken Joint,Office,Discount Store
94,Woodland Park,Pizza Place,Fast Food Restaurant,Discount Store,Grocery Store,Chinese Restaurant,Pharmacy,BBQ Joint,Sandwich Place,Fried Chicken Joint,Ice Cream Shop
95,Woodlawn,Rental Car Location,Discount Store,Coffee Shop,Fast Food Restaurant,Airport Service,Grocery Store,Airport Terminal,Park,BBQ Joint,Burger Joint
96,Wylam,Discount Store,Southern / Soul Food Restaurant,American Restaurant,Post Office,Social Club,Fast Food Restaurant,Shopping Mall,Bar,Fried Chicken Joint,Nightclub


## Attempt To See Clustering of the Venues
I am applying k-means clustering here.  This is an unsupervised machine learning algorithm that can help you discover underlying patterns in data.  It will create a number of clusters based on the value of k you set.  I initially set the value at five but ultimately dropped the value to two as I saw little differentiation when I had too many clusters.  At K=2, I saw a correlation of locally owned venues in one cluster versus people frequenting national chains like fast food restaurants and discount stores in the other cluster.

In [16]:
# set number of clusters
kclusters = 2

bhm_grouped_clustering = bhm_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bhm_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [17]:
#add the cluster labels 
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [18]:
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Acipco-Finley,Fried Chicken Joint,Discount Store,Fast Food Restaurant,Southern / Soul Food Restaurant,Intersection,Gas Station,Theater,Café,Brazilian Restaurant,Cosmetics Shop
1,0,Airport Highlands,Fast Food Restaurant,Rental Car Location,Airport Service,Café,Airport,American Restaurant,Park,Coffee Shop,Storage Facility,Fried Chicken Joint
2,0,Apple Valley,Discount Store,Pharmacy,Gas Station,Fast Food Restaurant,Breakfast Spot,Pizza Place,BBQ Joint,Sandwich Place,Southern / Soul Food Restaurant,Bookstore
3,0,Arlington - West End,Discount Store,Fast Food Restaurant,Pharmacy,Grocery Store,Fried Chicken Joint,Gas Station,Park,Sandwich Place,Pizza Place,Chinese Restaurant
4,0,Belview Heights,Fast Food Restaurant,Discount Store,Fried Chicken Joint,Gas Station,American Restaurant,Grocery Store,Pharmacy,Southern / Soul Food Restaurant,Sandwich Place,Pizza Place
...,...,...,...,...,...,...,...,...,...,...,...,...
93,0,West Goldwire,Gas Station,Trail,Grocery Store,Construction & Landscaping,Sandwich Place,Dog Run,Bar,Fried Chicken Joint,Office,Discount Store
94,0,Woodland Park,Pizza Place,Fast Food Restaurant,Discount Store,Grocery Store,Chinese Restaurant,Pharmacy,BBQ Joint,Sandwich Place,Fried Chicken Joint,Ice Cream Shop
95,0,Woodlawn,Rental Car Location,Discount Store,Coffee Shop,Fast Food Restaurant,Airport Service,Grocery Store,Airport Terminal,Park,BBQ Joint,Burger Joint
96,0,Wylam,Discount Store,Southern / Soul Food Restaurant,American Restaurant,Post Office,Social Club,Fast Food Restaurant,Shopping Mall,Bar,Fried Chicken Joint,Nightclub


In [19]:
bhm_merged = bhm_neighborhoods_geo
bhm_merged = bhm_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='NAME')
bhm_merged = bhm_merged[bhm_merged['Cluster Labels'].notnull()]
bhm_merged

,FID,NUM,NAME,Shape_Leng,Shape_Area,geometry,lng,lat,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,701,Apple Valley,57024.688015,2.008130e+07,"POLYGON ((-86.68047 33.67689, -86.68031 33.676...",-86.686467,33.666536,0.0,Discount Store,Pharmacy,Gas Station,Fast Food Restaurant,Breakfast Spot,Pizza Place,BBQ Joint,Sandwich Place,Southern / Soul Food Restaurant,Bookstore
2,3,705,Sun Valley,36022.229835,1.988828e+07,"MULTIPOLYGON (((-86.70463 33.64090, -86.70817 ...",-86.705592,33.653732,0.0,Discount Store,Gas Station,Pizza Place,Pharmacy,Fried Chicken Joint,BBQ Joint,Construction & Landscaping,Sandwich Place,Breakfast Spot,Southern / Soul Food Restaurant
3,4,1003,Spring Lake,97411.325741,5.917413e+07,"POLYGON ((-86.65253 33.60168, -86.65271 33.601...",-86.668153,33.616183,0.0,Fast Food Restaurant,Mexican Restaurant,Shoe Store,Fried Chicken Joint,Mobile Phone Shop,Furniture / Home Store,Sandwich Place,American Restaurant,Grocery Store,Discount Store
4,5,703,Echo Highlands,111881.645356,1.023235e+08,"MULTIPOLYGON (((-86.71446 33.63031, -86.71515 ...",-86.724420,33.630341,0.0,Gas Station,Sandwich Place,Discount Store,Bookstore,Burger Joint,Moving Target,Donut Shop,Food Court,Food,Flower Shop
5,6,1105,Hooper City,147831.568931,1.942805e+08,"POLYGON ((-86.84833 33.54339, -86.84837 33.543...",-86.854013,33.587894,0.0,Construction & Landscaping,Home Service,Business Service,Locksmith,Grocery Store,Shipping Store,Dance Studio,Dive Bar,Flower Shop,Fast Food Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,95,902,Hillman,10058.735701,3.087973e+06,"POLYGON ((-86.90040 33.44693, -86.90089 33.446...",-86.904372,33.446496,0.0,Discount Store,Fast Food Restaurant,Gas Station,Pharmacy,Grocery Store,Park,Seafood Restaurant,Trail,Fried Chicken Joint,Sandwich Place
95,96,2203,Roosevelt,35175.041401,2.169545e+07,"POLYGON ((-86.92949 33.42696, -86.93072 33.427...",-86.924966,33.439668,0.0,Fast Food Restaurant,Discount Store,Grocery Store,Gas Station,Pharmacy,Intersection,Seafood Restaurant,Fried Chicken Joint,Deli / Bodega,Bowling Alley
96,97,903,Hillman Park,5680.082511,1.259158e+06,"POLYGON ((-86.90283 33.44264, -86.90283 33.442...",-86.902127,33.444293,0.0,Fast Food Restaurant,Discount Store,Pharmacy,Grocery Store,Gas Station,Park,Seafood Restaurant,Fried Chicken Joint,Trail,Post Office
97,98,2202,West Brownville,17529.680219,6.166299e+06,"POLYGON ((-86.90539 33.43528, -86.90777 33.438...",-86.909527,33.440755,0.0,Discount Store,Fast Food Restaurant,Pharmacy,Grocery Store,Gas Station,Seafood Restaurant,Fried Chicken Joint,Trail,Office,Breakfast Spot


In [20]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12,
               zoom_control=False,
               scrollWheelZoom=False,
               dragging=False)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bhm_merged['lat'], bhm_merged['lng'], bhm_merged['NAME'], bhm_merged['Cluster Labels'].astype('int32')):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Conclusions from the Map
The purple cluster (we'll call that cluster one), is dominated by locally owned venues.  If you look at the map of Urban Renewal Districts, those overlay cluster one with a high degree of correlation.  The second cluster (represented in red) are dominated by venues that are not locally owned.  Where there is close proximity between red and purple, it would seem to me that there is opportunity to expand the purple cluster by taking advantage of the growth brought about by the revitalization.

<style>.embed-container {position: relative; padding-bottom: 80%; height: 0; max-width: 100%;} .embed-container iframe, .embed-container object, .embed-container iframe{position: absolute; top: 0; left: 0; width: 100%; height: 100%;} small{position: absolute; z-index: 40; bottom: 0; margin-bottom: -15px;}</style><div class="embed-container"><iframe width="500" height="400" frameborder="0" scrolling="no" marginheight="0" marginwidth="0" title="Urban Renewal Districts" src="//birmingham.maps.arcgis.com/apps/Embed/index.html?webmap=8ca77ce4e0af430d8a66016f98898491&extent=-87.0026,33.475,-86.6593,33.6295&zoom=true&previewImage=false&scale=true&disable_scroll=true&theme=light"></iframe></div>

## Cluster Breakdown
What is this clustering of venues really tell us.  In order to see that it is helpful to break the cluster out and see the venue rankings within each cluster.  This is the detail information for each cluster.

### Cluster One - Red Markers On Map - Dominated by Venues Not Locally Owned

In [21]:
bhm_merged.loc[bhm_merged['Cluster Labels'] == 0, bhm_merged.columns[[2] + list(range(8, bhm_merged.shape[1]))]]

,NAME,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Apple Valley,0.0,Discount Store,Pharmacy,Gas Station,Fast Food Restaurant,Breakfast Spot,Pizza Place,BBQ Joint,Sandwich Place,Southern / Soul Food Restaurant,Bookstore
2,Sun Valley,0.0,Discount Store,Gas Station,Pizza Place,Pharmacy,Fried Chicken Joint,BBQ Joint,Construction & Landscaping,Sandwich Place,Breakfast Spot,Southern / Soul Food Restaurant
3,Spring Lake,0.0,Fast Food Restaurant,Mexican Restaurant,Shoe Store,Fried Chicken Joint,Mobile Phone Shop,Furniture / Home Store,Sandwich Place,American Restaurant,Grocery Store,Discount Store
4,Echo Highlands,0.0,Gas Station,Sandwich Place,Discount Store,Bookstore,Burger Joint,Moving Target,Donut Shop,Food Court,Food,Flower Shop
5,Hooper City,0.0,Construction & Landscaping,Home Service,Business Service,Locksmith,Grocery Store,Shipping Store,Dance Studio,Dive Bar,Flower Shop,Fast Food Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
94,Hillman,0.0,Discount Store,Fast Food Restaurant,Gas Station,Pharmacy,Grocery Store,Park,Seafood Restaurant,Trail,Fried Chicken Joint,Sandwich Place
95,Roosevelt,0.0,Fast Food Restaurant,Discount Store,Grocery Store,Gas Station,Pharmacy,Intersection,Seafood Restaurant,Fried Chicken Joint,Deli / Bodega,Bowling Alley
96,Hillman Park,0.0,Fast Food Restaurant,Discount Store,Pharmacy,Grocery Store,Gas Station,Park,Seafood Restaurant,Fried Chicken Joint,Trail,Post Office
97,West Brownville,0.0,Discount Store,Fast Food Restaurant,Pharmacy,Grocery Store,Gas Station,Seafood Restaurant,Fried Chicken Joint,Trail,Office,Breakfast Spot


### Cluster Two - Purple Markers On Map - Dominated by Locally Owned Venues

In [22]:
bhm_merged.loc[bhm_merged['Cluster Labels'] == 1, bhm_merged.columns[[2] + list(range(8, bhm_merged.shape[1]))]]

,NAME,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,East Birmingham,1.0,Coffee Shop,Discount Store,BBQ Joint,Bar,Fried Chicken Joint,Airport Terminal,American Restaurant,Pizza Place,Burger Joint,Brewery
38,Norwood,1.0,American Restaurant,Bar,Brewery,Coffee Shop,Theater,BBQ Joint,Park,Fried Chicken Joint,Hotel,Sandwich Place
39,Evergreen,1.0,Fried Chicken Joint,Bar,American Restaurant,Theater,Coffee Shop,Hotel,Gas Station,Cocktail Bar,Fast Food Restaurant,Park
44,Druid Hills,1.0,American Restaurant,Hotel,Brewery,Coffee Shop,Bar,Theater,Southern / Soul Food Restaurant,Fried Chicken Joint,Park,Lounge
46,East Avondale,1.0,Park,Grocery Store,Coffee Shop,BBQ Joint,Bar,Pizza Place,Brewery,American Restaurant,Sandwich Place,Discount Store
47,North Avondale,1.0,Coffee Shop,Brewery,American Restaurant,Bar,BBQ Joint,Sandwich Place,Hotel,Fried Chicken Joint,Italian Restaurant,Pub
48,Enon Ridge,1.0,Fried Chicken Joint,Bar,American Restaurant,Theater,Hotel,Discount Store,Fast Food Restaurant,Cocktail Bar,Coffee Shop,Park
49,Fountain Heights,1.0,Bar,Fried Chicken Joint,Brewery,Coffee Shop,Pizza Place,Southern / Soul Food Restaurant,Hotel,Mexican Restaurant,Fast Food Restaurant,Park
50,Central City,1.0,Bar,Brewery,Coffee Shop,Park,Pizza Place,American Restaurant,Hotel,Mexican Restaurant,Theater,BBQ Joint
51,Crestwood South,1.0,Pizza Place,Grocery Store,Discount Store,Mexican Restaurant,Coffee Shop,Pharmacy,Bar,BBQ Joint,Italian Restaurant,Burger Joint
